In [1]:
!pip install pyDOE
!pip install cma

  Created wheel for pyDOE: filename=pyDOE-0.3.8-cp36-none-any.whl size=18178 sha256=02dafbfdc8065550ab5a30338b6cd875cdca9cbe7d90661ce81b1cd43e2f5fa1
  Stored in directory: /root/.cache/pip/wheels/7c/c8/58/a6493bd415e8ba5735082b5e0c096d7c1f2933077a8ce34544
Successfully built pyDOE
     |████████████████████████████████| 245kB 2.7MB/s 


In [0]:
import pyDOE
from scipy import stats
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import Bounds
from collections import namedtuple
import cma
from sklearn.preprocessing import MinMaxScaler
import scipy.stats.distributions as dist
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.gaussian_process.kernels import DotProduct
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.linear_model import ElasticNet
import bbobbenchmarks as bn
import sklearn
from sklearn.preprocessing import PolynomialFeatures

## Surrogate Models & Helper Functions

In [0]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' F16 '''
def F16(X):
    f = bn.F16()
    X = np.array(X)
    return f(X)

''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs, dim):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=dim, samples=n_obs, criterion='m')
    X = [lhd[:,idx] for idx in range(dim)]
    return X

def create_basis_function(data):
    true = np.array(data['Y'])
    data = pd.DataFrame(np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(data.iloc[:,:-1])))
    data['Y'] = pd.Series(true)
    return data


''' Create Basis Functions '''
def create_function_basis(x):
    return np.atleast_2d(PolynomialFeatures(degree=2).fit_transform(x.reshape(1,-1)))


''' Elastic Net Regression '''
def elastic_net(train_data,test_data):
    scaler =  MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    regr = ElasticNet(alpha= 0.7403287317239838 ,random_state=0 , l1_ratio=0.5353535353535354, fit_intercept =True, max_iter=3000,selection='random').fit(scaler.transform ( np.array(train_data.iloc[:,:-1])) ,  np.array(train_data.iloc[:,-1]))
    pred = regr.predict(scaler.transform(test_data))
    def predict(scaler, regr):
        def __predict__(x):
            x = create_function_basis(x)
            return regr.predict(scaler.transform(x))
        return __predict__
    return regr,pred, predict(scaler, regr)
    

''' Kriging'''
def kriging(train_data,test_data):
    kernel =  RBF()
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer= 15,random_state=0,
                                   normalize_y=True ).fit(scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)


''' Support Vector Regression'''
def _SVR(train_data,test_data):
    scaler = MinMaxScaler().fit(np.r_[train_data.iloc[:,:-1].values])
    gpr = sklearn.svm.SVR(kernel='rbf', gamma = 62.906077 , C = 0.000100 ,max_iter=1500).fit( scaler.transform(train_data.iloc[:,:-1]), train_data.iloc[:,-1])
    pred = gpr.predict(scaler.transform(test_data))
    def predict(scaler, gpr):
        def __predict__(x):
            x = np.atleast_2d(x)
            return gpr.predict(scaler.transform(x))
        return __predict__
    return gpr,pred, predict(scaler,gpr)

## Load Training and Test Data Set initially Generated

In [0]:
path = "train_16_1000Samples.csv"
train = pd.read_csv(path).iloc[:,1:]
test = pd.read_csv('test_16_200Samples.csv').iloc[:,1:]
true = np.array(test['Y'])

## Surrogate Models

In [0]:
model_kri , pred_kri , predict_kri = kriging(train,test.iloc[:,:-1])
model_svr , pred_svr , predict_svr = _SVR(train,test.iloc[:,:-1])
train = create_basis_function(train)
test = create_basis_function(test)
model_eln , pred_eln , predict_eln = elastic_net(train,test.iloc[:,:-1])

## CMA-ES

In [6]:
Columns = ['Kri' , 'SVR' , 'ELN' ]
Cols = []
for j in range(len(Columns)):
    for i in range(1,51):
        Cols.append(Columns[j]+'_X'+str(i))
const = [ [-5] * 50, [5] * 50 ]

opt = cma.CMAOptions()
opt.set("bounds", const)
opt.set ("seed" , 0)
opt.set ("maxfevals" , 50000)

n_obs , dim =  30, 50
G = DOE(n_obs, dim)
G  = [ linearscaletransform(G[idx] , range_out=(-5,5)) for idx in range(dim) ]
G = [ G[idx].reshape(n_obs,1) for idx in range(len(G)) ]
X_Values = np.zeros([30,150])
for i in range(G[1].shape[0]):
        min_kri = cma.fmin(predict_kri , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_svr = cma.fmin(predict_svr , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        min_eln = cma.fmin(predict_eln , np.concatenate(G, 1)[i] , 2.5 , options=opt) [0]
        X_Values [i,:] = list(min_kri)+list(min_svr)+list(min_eln)

(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=996878, Mon Apr 13 20:10:13 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.796111250460871e+02 1.0e+00 2.38e+00  2e+00  2e+00 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 20:10:13 2020)
final/bestever f-value = -1.796111e+02 -1.796111e+02
incumbent solution: [-4.9450084   4.19307946 -3.98529355  0.62585121  2.92621581  1.46438873
  3.24856702 -1.65669653 ...]
std deviations: [2.37869935 2.37793491 2.37664748 2.37671534 2.38317726 2.37730322
 2.38086256 2.38234913 ...]
(7_w,15)-aCMA-ES (mu_w=4.5,w_1=34%) in dimension 50 (seed=831715, Mon Apr 13 20:10:14 2020)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     15 -1.807846610046242e+02 1.0e+00 2.36e+00  2e+00  2e+00 0:00.0
termination on tolfun=1e-11 (Mon Apr 13 20:10:14 2020)
final/bestever f-value = -1.807847e+02 -1.807847e+02
incumbent solution: [-4.4745329   4.37145565 -3.72550942  2.20084301  2.41

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61112505] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.38e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.38e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

    3     45 -1.796111059562073e+02 1.1e+00 2.26e+00  2e+00  2e+00 0:00.1
  100   1500 -1.796117986756367e+02 1.9e+00 2.15e+00  2e+00  2e+00 0:02.0
  200   3000 -1.796118009696420e+02 3.1e+00 1.59e+00  8e-01  2e+00 0:03.9
  300   4500 -1.796118009832446e+02 4.4e+00 1.75e+00  7e-01  2e+00 0:05.9
  400   6000 -1.796118007243084e+02 6.3e+00 1.46e+00  4e-01  2e+00 0:07.9
  500   7500 -1.796118009835298e+02 8.9e+00 1.13e+00  2e-01  1e+00 0:09.8
  600   9000 -1.796118009827230e+02 1.3e+01 1.06e+00  1e-01  1e+00 0:11.8
  700  10500 -1.796118009711001e+02 1.9e+01 7.27e-01  7e-02  9e-01 0:13.7
  800  12000 -1.796118009726552e+02 2.7e+01 1.09e+00  7e-02  1e+00 0:15.7
  900  13500 -1.796118009649785e+02 3.8e+01 1.13e+00  5e-02  1e+00 0:17.7
 1000  15000 -1.796118009836204e+02 6.0e+01 1.01e+00  3e-02  1e+00 0:19.7
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.796118009833861e+02 8.7e+01 1.04e+00  3e-02  1e+00 0:21.6
 1200  18000 -1.796118009835920e+02 1.2e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61112505] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.36e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.796118008243400e+02 1.9e+00 1.42e+00  1e+00  1e+00 0:01.9
  200   3000 -1.796118009818597e+02 2.8e+00 1.24e+00  6e-01  1e+00 0:03.9
  300   4500 -1.796118008955844e+02 4.0e+00 9.69e-01  4e-01  1e+00 0:05.8
  400   6000 -1.796118009618855e+02 6.3e+00 1.39e+00  4e-01  2e+00 0:07.8
  500   7500 -1.796118009654124e+02 9.2e+00 8.11e-01  2e-01  9e-01 0:09.7
  600   9000 -1.796118009278402e+02 1.3e+01 9.30e-01  1e-01  1e+00 0:11.7
  700  10500 -1.796118009836603e+02 2.0e+01 1.28e+00  1e-01  1e+00 0:13.7
  800  12000 -1.796118009820492e+02 2.8e+01 7.38e-01  5e-02  9e-01 0:15.6
  900  13500 -1.796118009831520e+02 4.1e+01 7.08e-01  3e-02  9e-01 0:17.6
 1000  15000 -1.796118009836545e+02 5.8e+01 3.59e-01  1e-02  4e-01 0:19.5
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.796118009836108e+02 8.4e+01 2.87e-01  7e-03  4e-01 0:21.5
 1200  18000 -1.796118009836554e+02 1.2e+02 2.96e-01  5e-03  4e-01 0:23.4
 1300  19500 -1.796118009836503e+02 1.7e

...

  100   1500 -1.796117945843057e+02 1.9e+00 1.98e+00  1e+00  2e+00 0:01.9
  200   3000 -1.796117953129174e+02 2.7e+00 1.66e+00  8e-01  2e+00 0:03.8
  300   4500 -1.796118005611307e+02 4.2e+00 1.29e+00  5e-01  1e+00 0:05.6
  400   6000 -1.796118009365422e+02 6.1e+00 1.34e+00  3e-01  1e+00 0:07.5
  500   7500 -1.796118009686714e+02 9.0e+00 1.33e+00  2e-01  1e+00 0:09.4
  600   9000 -1.796118009795733e+02 1.3e+01 1.69e+00  2e-01  2e+00 0:11.3
  700  10500 -1.796118009782935e+02 1.9e+01 1.34e+00  1e-01  2e+00 0:13.2
  800  12000 -1.796118009835912e+02 2.7e+01 6.98e-01  5e-02  8e-01 0:15.1
  900  13500 -1.796118009825291e+02 3.7e+01 8.70e-01  4e-02  1e+00 0:17.0
 1000  15000 -1.796118009836496e+02 5.3e+01 5.31e-01  2e-02  6e-01 0:18.9
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.796118009835944e+02 7.8e+01 6.07e-01  1e-02  7e-01 0:20.8
 1200  18000 -1.796118009835738e+02 1.1e+02 3.81e-01  7e-03  4e-01 0:22.7
 1300  19500 -1.796118009836602e+02 1.7e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61112505] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.796118001080145e+02 1.9e+00 1.38e+00  1e+00  1e+00 0:01.9
  200   3000 -1.796118000806208e+02 3.1e+00 1.43e+00  7e-01  2e+00 0:03.8
  300   4500 -1.796118007035685e+02 4.5e+00 1.13e+00  4e-01  1e+00 0:05.7
  400   6000 -1.796118005473931e+02 6.2e+00 1.35e+00  3e-01  2e+00 0:07.6
  500   7500 -1.796118009752181e+02 8.8e+00 1.19e+00  2e-01  1e+00 0:09.6
  600   9000 -1.796118009823001e+02 1.3e+01 1.31e+00  2e-01  2e+00 0:11.5
  700  10500 -1.796118009836455e+02 1.8e+01 1.43e+00  1e-01  2e+00 0:13.4
  800  12000 -1.796118009826352e+02 2.6e+01 9.20e-01  6e-02  1e+00 0:15.3
  900  13500 -1.796118009834864e+02 3.9e+01 1.06e+00  5e-02  1e+00 0:17.3
 1000  15000 -1.796118009835875e+02 6.0e+01 6.66e-01  2e-02  8e-01 0:19.2
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.796118009836597e+02 8.8e+01 6.11e-01  1e-02  7e-01 0:21.1
 1200  18000 -1.796118009836603e+02 1.2e+02 2.47e-01  4e-03  3e-01 0:23.0
 1300  19500 -1.796118009836369e+02 1.6e

/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: function_values is not a list of scalars,
                        the first element equals [-179.61112505] with non-scalar type <class 'numpy.ndarray'>.
                        Using now ``[v[0] for v in function_values]`` instead (further warnings are suppressed) ()
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')
/usr/local/lib/python3.6/dist-packages/cma/utilities/utils.py:333: UserWarning: flat fitness (f=-179.611125, sigma=2.35e+00).
                    For small sigma, this could indicate numerical convergence.
                    Otherwise, please (re)consider how to compute the fitness more elaborately. (iteration=1)
  ')')

  100   1500 -1.796118000866801e+02 1.9e+00 1.17e+00  8e-01  1e+00 0:01.9
  200   3000 -1.796118006322483e+02 3.2e+00 1.36e+00  7e-01  2e+00 0:03.8
  300   4500 -1.796118001024118e+02 5.0e+00 1.53e+00  5e-01  2e+00 0:05.7
  400   6000 -1.796118009213683e+02 7.0e+00 8.75e-01  2e-01  1e+00 0:07.5
  500   7500 -1.796118009833017e+02 1.0e+01 7.40e-01  1e-01  9e-01 0:09.4
  600   9000 -1.796118009836434e+02 1.5e+01 4.52e-01  6e-02  5e-01 0:11.4
  700  10500 -1.796118009830360e+02 2.1e+01 2.66e-01  2e-02  3e-01 0:13.2
  800  12000 -1.796118009833889e+02 2.9e+01 2.35e-01  1e-02  3e-01 0:15.2
  900  13500 -1.796118009836592e+02 4.0e+01 2.64e-01  1e-02  3e-01 0:17.1
 1000  15000 -1.796118009836599e+02 5.8e+01 1.82e-01  6e-03  2e-01 0:19.0
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
 1100  16500 -1.796118009836571e+02 8.4e+01 1.02e-01  2e-03  1e-01 0:20.9
 1200  18000 -1.796118009836591e+02 1.2e+02 8.88e-02  1e-03  1e-01 0:22.8
 1300  19500 -1.796118009836590e+02 1.8e

In [7]:
X_Values = pd.DataFrame(X_Values)
X_Values.columns = Cols
X_Values.to_csv('Results\\F16_X_Values.csv')
Krig_Fun = np.zeros(30)
SVM_Fun = np.zeros(30)
ELN_Fun = np.zeros(30)
for i in range(X_Values.shape[0]):
    Krig_Fun [i] = F16(X_Values.iloc[i,:50])
    SVM_Fun [i] = F16(X_Values.iloc[i,50:100])
    ELN_Fun [i] = F16(X_Values.iloc[i,100:150])
print ('Kriging')
print (stats.mode(Krig_Fun) , np.std(Krig_Fun))
print ('SVM')
print (stats.mode(SVM_Fun) , np.std(SVM_Fun))
print ('ELN')
print (stats.mode(ELN_Fun) , np.std(ELN_Fun))

Kriging
ModeResult(mode=array([-208.43774986]), count=array([1])) 16.414655791461612
SVM
ModeResult(mode=array([-200.86332618]), count=array([1])) 13.48030835616415
ELN
ModeResult(mode=array([-201.20003515]), count=array([1])) 14.92352002213522
